Copyright (c) Microsoft Corporation. All rights reserved.
Licensed under the MIT License.
Modified by Shohei Nagata.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-horovod/distributed-pytorch-with-horovod.png)

# PyTorchモデルでの画像分類
本日のハンズオンでは[MNIST](http://yann.lecun.com/exdb/mnist/)データセットを対象に画像分類を行うPyTorchモデルを学習します。

## Prerequisites
Azure Machine Learning Notebook VMを使用している場合は、すべての設定が完了しています。
GPUインスタンスを使用する場合は事前にクォーター申請が必要となる場合があります。

参考：[Azure Machine Learning を使用するリソースのクォータの管理と引き上げ](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-manage-quotas)、
[申請手順](https://docs.microsoft.com/ja-jp/azure/azure-portal/supportability/regional-quota-requests#request-a-quota-increase-by-region-from-help--support)

In [ ]:
# Azure ML SDKのバージョン確認
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

## ワークスペースの設定
Azure ML ワークスペースによってAzure MLで使用するアセット類 (データ、スクリプト、出力、等々)を管理していきます。
![](https://docs.microsoft.com/ja-jp/azure/machine-learning/media/concept-azure-machine-learning-architecture/architecture.svg)

`Workspace.from_config()` は、`config.json` に格納された詳細情報から、ワークスペース・オブジェクトを作成します。   

事前にAzure ML Studioから構成ファイル (config.json)をダウンロードし、本スクリプトと同一階層に置きます。  

初回実行時は認証を行う必要があるため、実行結果部分の指示に従って https://microsoft.com/devicelogin にアクセスし、認証コードを入力します。

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## 計算環境の準備

モデルをトレーニングするためには、[コンピューティング先](hhttps://docs.microsoft.com/ja-jp/azure/machine-learning/concept-azure-machine-learning-architecture#computes)を作成する必要があります。このノートブックでは、コンピューティング クラスターをリモートトレーニング用のコンピュートリソースとして使用します。  
具体的には，以下のコードで，`STANDARD_NC6`のGPUクラスターを作成し，`0`から`2`のノードにオートスケールします。

> Azure ML データサイエンティストのロールのみを持っている場合、コンピュートリソースを作成する権限はありませんのでご注意ください。このセクションで説明するコンピュートターゲットがまだ存在していない場合は、ワークスペースまたはIT管理者に相談して作成してください。

**コンピューティングクラスターの作成には約5分かかります。** 同一名称のものがワークスペースにある場合、下記コードは作成プロセスをスキップします。



In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster-ds13"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS13_v2',
                                                           max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

上記のコードでは，GPUコンピュートを作成しています．代わりにCPUコンピュートを作成したい場合は、`Standard_DS13_v2`のように、`vm_size`パラメータに異なるVMサイズを指定してください。

## リモートでのモデル学習 
リモート (Azure ML上の)計算環境が準備できたので、学習ジョブを実行してみましょう。

### プロジェクトディレクトリの作成
ローカルマシンからリモートリソースにアクセスするために必要なコードをすべて格納するディレクトリを作成します。このディレクトリには、トレーニングスクリプトと、トレーニングスクリプトが依存する追加ファイルが含まれます。

In [ ]:
import os

project_folder = './pytorch-mnist'
os.makedirs(project_folder, exist_ok=True)

### トレーニングスクリプトの準備
ここでは、トレーニング用のスクリプトを作成します。本コンテンツでは、MNISTの分散学習用のスクリプトを`pytorch_mnist.py`で用意しています。  
(これは [PyTorch公式のMNISTサンプル](https://github.com/pytorch/examples/blob/master/mnist/main.py)をベースにログ記録を追加したものです)  
実際には、カスタムのPyTorchトレーニングスクリプトをそのまま使用して、コードを変更することなくAzure MLで実行することが可能です。

Azure MLの[ログ記録](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-azure-machine-learning-architecture#logging)機能を使いたい場合は、トレーニングスクリプト内に少量のAzure MLロジックを追加する必要があります。この例では、ロギング間隔ごとに、そのミニバッチの損失をAzure ML Runにロギングします。

これを行うために、`pytorch_mnist.py`では、まずスクリプト内にAzure MLの`Run`オブジェクトへのアクセスを追加しています。
```Python
from azureml.core.run import Run
run = Run.get_context()
```
そしてスクリプトの後半で、損失メトリックを Run に記録します。
```Python
run.log('loss', loss.item())
```

トレーニングスクリプト `pytorch_mnist.py`をプロジェクトディレクトリ内へコピーします。

In [ ]:
import shutil

shutil.copy('pytorch_mnist.py', project_folder)

### 実験の作成
[実験 (Experiment)](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment)を作成して、このPyTorchチュートリアルのワークスペースでのすべての実行を追跡します。

In [ ]:
from azureml.core import Experiment

experiment_name = 'pytorch-mnist'
experiment = Experiment(ws, name=experiment_name)

### 環境の作成

Azure MLではいくつかの[キュレートされた実行環境](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-environments#use-a-curated-environment)が用意されています。
今回はPyTorch 1.6 GPU環境を使用します。こちらのキュレートされた環境には今回のトレーニングスクリプトで必要なtorch, torchvisionも含まれています。

参考：[キュレーションされた環境一覧](https://docs.microsoft.com/ja-jp/azure/machine-learning/resource-curated-environments)

In [ ]:
from azureml.core import Environment

pytorch_env = Environment.get(ws, name='AzureML-PyTorch-1.6-GPU')

In [ ]:
# 中身の確認
print(pytorch_env.python.conda_dependencies.serialize_to_string())

### トレーニングジョブの設定
`ScriptRunConfig`を用いてスクリプトの実行構成を作成します。
![](../img/scriptrunconfig.png)

参考：[スクリプトの実行構成を作成する](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-set-up-training-targets#create-the-script-run-configuration)

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='pytorch_mnist.py',
                      compute_target=compute_target,
                      environment=pytorch_env)

### トレーニングジョブの実行 (送信)
前項の`ScriptRunConfig`で設定した条件に基づいて実験を実行 (送信)します。

In [ ]:
run = experiment.submit(src)
print(run)

### モニタリング
Jupyterウィジェットを使って実行の進捗状況を監視することができます。実行のサブミッションと同様に、ウィジェットは非同期で、ジョブが完了するまで10～15秒ごとに自動で更新されます。ウィジェットでは、Azure MLの実行に記録した損失指標が自動的に表示・可視化されます。

※VSCode上で実行する場合、テーマ設定 (背景色)によってはAzure MLウィジェットが見えにくくなる可能性があります。その場合はLightテーマの使用をお勧めします。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
run.wait_for_completion(show_output=True) # this provides a verbose log